# LL(1) Parser

In [2]:
Grammar = {"S": ["aBDh"],"B":["cC"],"C":["bC","#"],"D":["EF"],"E":["g","#"],"F":["f","#"]}

In [3]:
Follow = {}
First = {}

def find_first(key):
    # if key in First.keys():
    #     return First[key]
    # else:
        for rule in Grammar[key]:
            if rule == "#":
                First[key].add("#")
            elif rule[0].isupper():
                First[key] = First[key].union(find_first(rule[0]))
                if "#" in First[key]:
                    First[key] = First[key].union(find_first(rule[1:]))
            else:
                First[key].add(rule[0])
        return First[key]
for key in Grammar.keys():
    Follow[key] = set()
    First[key] = set()
    # First[key] = find_first(key)

for key in Grammar.keys():
    First[key] = find_first(key)
find_first("D")

{'#', 'f', 'g'}

In [4]:
def find_follow(key, start="S"):
    for k in Grammar.keys():
        if k == start:
            Follow[k].add("$")
        for rule in Grammar[k]:
            if key in rule:
                index = rule.find(key)
                if index == len(rule) - 1: #last symbol
                    if k != key:
                        Follow[key] = Follow[key].union(find_follow(k))
                else:
                    if rule[index + 1].isupper():#next symbol is non-terminal
                        temp = First[rule[index + 1]]
                        idx=1
                        # print("here",temp)
                        while "#" in temp:
                            # print("here",temp)
                            temp.remove("#")
                            Follow[key] = Follow[key].union(temp)
                            if index + 1+idx < len(rule):
                                if rule[index + 1+idx].isupper():
                                # print(k, rule[index + 1+idx], temp, Follow[key])
                                    temp = First[rule[index + idx+1]]
                                    idx+=1
                                else: 
                                    temp.add(rule[index + 1+idx])
                                    break
                            else:
                                temp = temp.union(find_follow(k))
                        Follow[key] = Follow[key].union(temp)
                        # else:
                        #     Follow[key] = Follow[key].union(temp)
                    else:
                        Follow[key].add(rule[index + 1])
    return Follow[key]
# find_follow("E")


for i in Grammar.keys():
    print("follow of : ", i, " is ", find_follow(i))

follow of :  S  is  {'$'}
follow of :  B  is  {'g', 'f', 'h'}
follow of :  C  is  {'g', 'f', 'h'}
follow of :  D  is  {'h'}
follow of :  E  is  {'f', 'h'}
follow of :  F  is  {'h'}


In [5]:
# for creating parsing table
import pandas as pd
# fill the columns head with terminals and $
col_head = set()
for i in Grammar.keys():
    col_head = col_head.union(First[i])
    col_head = col_head.union(Follow[i])
col_head = list(col_head)
col_head.remove("#")
# col_head.append("$")
col_head.sort()
# print(col_head)
col_head

['$', 'a', 'b', 'c', 'f', 'g', 'h']

In [6]:
# row index with non-terminals
row_index = list(Grammar.keys())
row_index.sort()
 
row_index

['B', 'C', 'D', 'E', 'F', 'S']

In [7]:
# create a dataframe with row and column
table = pd.DataFrame(index=row_index, columns=col_head)
table

,$,a,b,c,f,g,h
B,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# fill the table with rules
for i in Grammar.keys():
    for rule in Grammar[i]:
        if rule == "#":
            for j in Follow[i]:
                table.loc[i,j] = rule
        else:
            for j in First[i]:
                table.loc[i,j] = rule
table


,$,a,b,c,f,g,h,#
B,NaN,NaN,NaN,cC,NaN,NaN,NaN,NaN
C,NaN,NaN,bC,NaN,#,#,#,bC
D,NaN,NaN,NaN,NaN,EF,EF,EF,NaN
E,NaN,NaN,NaN,NaN,#,g,#,g
F,NaN,NaN,NaN,NaN,f,NaN,#,NaN
S,NaN,aBDh,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# initialize the predictive parsing table
table = {}



# fill the table with rules
for non_terminal in Grammar:
    for symbol in First[non_terminal]:
        if symbol != '#':
            table[(non_terminal, symbol)] = Grammar[non_terminal][0]
    if '#' in First[non_terminal]:
        for symbol in Follow[non_terminal]:
            table[(non_terminal, symbol)] = '#'

In [7]:
import pandas as pd

# create a list of tuples from the table dictionary
table_list = [(str(key[0]), str(key[1]), str(value)) for key, value in table.items()]

# create a pandas DataFrame from the list of tuples
df = pd.DataFrame(table_list, columns=['Non-Terminal', 'Terminal', 'Production Rule'])

# display the DataFrame
print(df)

   Non-Terminal Terminal Production Rule
0             S        a            aBDh
1             B        c              cC
2             C        b              bC
3             C        g               #
4             C        f               #
5             C        h               #
6             D        g              EF
7             D        f              EF
8             D        h              EF
9             E        g               g
10            E        f               #
11            E        h               #
12            F        f               f


In [8]:
import pandas as pd

# create a list of non-terminal symbols and terminal symbols
non_terminals = sorted(set([key[0] for key in table.keys()]))
terminals = sorted(set([key[1] for key in table.keys()]))

# create an empty DataFrame with non-terminals as row headers and terminals as column headers
df = pd.DataFrame(index=non_terminals, columns=terminals)

# fill the DataFrame with production rules
for key, value in table.items():
    df.loc[key[0], key[1]] = value

# display the DataFrame
print(df)

      a    b    c    f    g    h
B   NaN  NaN   cC  NaN  NaN  NaN
C   NaN   bC  NaN    #    #    #
D   NaN  NaN  NaN   EF   EF   EF
E   NaN  NaN  NaN    #    g    #
F   NaN  NaN  NaN    f  NaN  NaN
S  aBDh  NaN  NaN  NaN  NaN  NaN
